# Import Libraries

In [3]:
from census import Census
from us import states
import pandas as pd 
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely import wkt
from mpl_toolkits.axes_grid1 import make_axes_locatable
%matplotlib notebook

In [4]:
engine = create_engine('postgresql://amy@localhost:5432/amy')
c = Census("be588bd861cbf7e0545af372027fc4980855f07e")

# Parameters

In [20]:
# San Francisco, CA
fips = states.CA.fips
county = '075'
prefix = 'sf'
distances = 'ca_sanfrancisco_distances'
poi = 'ca_sanfrancisco_poi'
parcels = 'ca_sanfrancisco_parcels'

# Import Census Data

## Median Household Income

In [30]:
# Table B19013: Median HH Income
# https://censusreporter.org/tables/B19013

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B19013)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [6]:
data.to_sql('{}_household_inc'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Ratio of Income to Poverty Level

In [31]:
# Table C17002: Ratio of Income to Poverty Level
# https://censusreporter.org/tables/C17002

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(C17002)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [13]:
data.to_sql('{}_poverty_ratio'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Limited English

In [57]:
# Table B16005: Nativity by Language Spoken at Home by Ability to Speak English
# https://censusreporter.org/tables/B16005

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B16005)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [43]:
data.to_sql('{}_limited_english'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## People of Color

In [59]:
# Table B03002: Hispanic or Latino Origin by Race
# https://censusreporter.org/tables/B03002

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B03002)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [61]:
data.to_sql('{}_poc'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Zero Vehicle Households

In [13]:
# Table B25044: Tenure by Vehicles Available
# https://censusreporter.org/tables/B25044

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B25044)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [15]:
data.to_sql('{}_veh'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Seniors 75 Years and Over

In [64]:
# Table B01001: Sex by Age
# https://censusreporter.org/tables/B01001

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B01001)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [65]:
data.to_sql('{}_age'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## People with Disability

In [66]:
# Table C18108: Age by Number of Disabilities
# https://censusreporter.org/tables/C18108

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(C18108)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [67]:
data.to_sql('{}_disability'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Single-Parent Family

In [68]:
# Table B11004: Family Type by Presence and Age of Related Children
# https://censusreporter.org/tables/B11004

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B11004)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [69]:
data.to_sql('{}_family'.format(prefix), con=engine, schema='iggy', if_exists='replace')

## Severely Rent-Burdened Household

In [70]:
# Table B25070: Gross Rent as a Percentage of Household Income
# https://censusreporter.org/tables/B25070

data = pd.DataFrame.from_dict(c.acs5.state_county_blockgroup('group(B25070)', '{}'.format(fips), '{}'.format(county), Census.ALL))

In [71]:
data.to_sql('{}_rent_burden'.format(prefix), con=engine, schema='iggy', if_exists='replace')

# Create Feature Layer

In [17]:
query = """
    DROP TABLE IF EXISTS iggy.{0}_feature_layer;
    CREATE TABLE iggy.{0}_feature_layer AS (
        WITH low_inc AS (
            select DISTINCT "GEO_ID"
            , split_part("GEO_ID",'US',2)::VARCHAR as geoid 
            , "C17002_001E" - "C17002_008E" as low_inc
            , ("C17002_001E" - "C17002_008E")/"C17002_001E"::FLOAT as perc_low_inc
            from iggy.sf_poverty_ratio spr 
            where 1=1
            and "C17002_001E" > 0
        )
        , older AS (
            select DISTINCT "GEO_ID"
            , split_part("GEO_ID",'US',2)::VARCHAR as geoid 
            , "B01001_023E" + "B01001_024E" + "B01001_025E" + "B01001_047E" + "B01001_048E" + "B01001_049E" 
                as older_75
            , ("B01001_023E" + "B01001_024E" + "B01001_025E" + "B01001_047E" + "B01001_048E" + "B01001_049E" )/
                "B01001_001E"::FLOAT as perc_older
            from iggy.sf_age sa 
            where 1=1
            and "B01001_001E" > 0
        )
        , veh AS (
            select DISTINCT "GEO_ID"
            , split_part("GEO_ID",'US',2)::VARCHAR as geoid 
            , "B25044_003E" + "B25044_010E" as zero_veh
            , ("B25044_003E" + "B25044_010E") / "B25044_001E"::FLOAT as perc_zero_veh
            from iggy.sf_veh sv 
            where 1=1
            and "B25044_001E" > 0
        )
        SELECT c.geoid
        , c.geom
        , o.older_75
        , o.perc_older
        , l.low_inc
        , l.perc_low_inc
        , v.zero_veh
        , v.perc_zero_veh
        , CASE WHEN l.perc_low_inc >= 0.3 AND o.perc_older >= .10 AND v.perc_zero_veh >= .15 
            THEN 1 ELSE 0 END as priority_area
        FROM cbg_california c
        LEFT JOIN older o 
            ON o.geoid = c.geoid
        LEFT JOIN low_inc l 
            ON l.geoid = c.geoid
        LEFT JOIN veh v
            ON v.geoid = c.geoid
        WHERE 1=1
        AND c.countyfp = '{1}'
    );
""".format(prefix, county)

engine.execute(query)

# Outside CBG

In [58]:
query = """
    DROP TABLE IF EXISTS iggy.{0}_outside_cbg;
    CREATE TABLE iggy.{0}_outside_cbg AS (
        select distinct p."_id" 
        , ST_setSRID(p."_geometry", 4326) as geom
        , p.cbg 
        from iggy.{1} d 
        left join iggy.{2} p 
            on p."_id" = d.id 
        left join iggy.{3} poi 
            on poi._source_id  = d."_source_id" 
        --inner join iggy.sf_feature_layer sfl 
        --    on sfl.geoid = p.cbg 
        --    and sfl.priority_area = 1
        where 1=1
        and poi.cbg <> p.cbg
        and d.source_category = 'is_urgent_health_care'
        and LOWER(p.cdl_majority_category) not like '%%open space%%'
        and LOWER(p.cdl_majority_category) not like '%%forest%%'
        and lower(p.zoning_description) not like '%%industrial%%'
        and (distance_in_meters::FLOAT/1609) < 0.25
    );
""".format(prefix, distances, parcels, poi)

engine.execute(query)

# Updating Layer with Proximity Data

In [59]:
query = """
    DROP TABLE IF EXISTS iggy.{0}_urgent_care_proximity;
    CREATE TABLE iggy.{0}_urgent_care_proximity AS (
        WITH parcels AS (
            select distinct p."_id" 
            , ST_setSRID(p."_geometry", 4326) as geom
            , p.cbg 
            from iggy.{1} d 
            left join iggy.{2} p 
                on p."_id" = d.id 
            left join iggy.{3} poi 
                on poi._source_id  = d."_source_id" 
            INNER JOIN iggy.sf_feature_layer sfl 
                on sfl.geoid = p.cbg 
                and sfl.priority_area = 1
            where 1=1
            and d.source_category = 'is_urgent_health_care'
            and LOWER(p.cdl_majority_category) not like '%%open space%%'
            and LOWER(p.cdl_majority_category) not like '%%forest%%'
            and lower(p.zoning_description) not like '%%industrial%%'
            and (distance_in_meters::FLOAT/1609) >= 0.25
        )
        select DISTINCT sfl.geoid
        , sfl.geom
        , count(distinct par."_id")/count(distinct p."_id")::FLOAT AS perc_parcels_far
        FROM iggy.{2} p 
        LEFT JOIN parcels par
            ON par."_id" = p."_id"
        INNER JOIN iggy.sf_feature_layer sfl 
            on sfl.geoid = p.cbg 
            and sfl.priority_area = 1
        where 1=1
            and LOWER(p.cdl_majority_category) not like '%%open space%%'
            and LOWER(p.cdl_majority_category) not like '%%forest%%'
            and lower(p.zoning_description) not like '%%industrial%%'
        GROUP BY 1,2
    );
""".format(prefix, distances, parcels, poi)

engine.execute(query)